## Importing the libraries

In [23]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime

In [24]:
stream= open("spotify/spotify.yaml")
spotify_details = yaml.safe_load(stream)
auth_manager = SpotifyClientCredentials(client_id=spotify_details['Client_id'],
                                        client_secret=spotify_details['client_secret'])
sp = spotipy.client.Spotify(auth_manager=auth_manager)

## Importing the dataset

- Reading the 1 million playlists and keeping only the unique track URIs for the content-based recommendation system.
- The first for loop Read the 1,000 JSON files one at a time.
- The second for loop is for getting only the unique track.

In [21]:
def loop_slices(path, num_slices=20):
    cnt = 0
    cnt1 = 0
    mpd_playlists = []
    unique_tracks= pd.DataFrame()
    filenames = os.listdir(path)
    for fname in tqdm(sorted(filenames, key=len)):
        if fname.startswith("mpd.slice.") and fname.endswith(".json"):
            cnt += 1
            fullpath = os.sep.join((path, fname))
            f = open(fullpath)
            js = f.read()
            f.close()
            print(js)
            current_slice = json.loads(js)
            # Create a list of all playlists
            for playlist in current_slice['playlists']:
                cnt1 +=1
                mpd_playlists.append(playlist)
                if cnt1 == 1000:
                    cnt1=0
                    temp = pd.DataFrame()
                    temp=pd.DataFrame(mpd_playlists)
                    temp=temp.explode('tracks')
                    temp=pd.DataFrame(temp['tracks'].apply(pd.Series))
                    unique_tracks=pd.concat([unique_tracks,temp],axis=0)
                    unique_tracks.drop_duplicates(subset=['track_uri'],inplace=True)
                    mpd_playlists = []
            if cnt == num_slices:
                break
    return unique_tracks
# Path where the json files are extracted
path = 'spotify_million_playlist_dataset/data/'

In [22]:
df = loop_slices(path, num_slices=1000)
df.to_csv('1m.csv')

  0%|                                                   | 0/910 [00:00<?, ?it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|                                         | 1/910 [00:22<5:35:58, 22.18s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  0%|                                         | 2/910 [00:46<5:55:27, 23.49s/it]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in orde

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
df

NameError: name 'df' is not defined

In [15]:
df["track_uri"] = df["track_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["artist_uri"] = df["artist_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])
df["album_uri"] = df["album_uri"].apply(lambda x: re.findall(r'\w+$', x)[0])

KeyError: 'track_uri'

In [25]:
df=pd.read_csv('Data/1mV3.csv') #Dropped all columns except ['track_uri', 'artist_uri', 'album_uri']

In [26]:
df.columns

Index(['Unnamed: 0', 'track_uri', 'artist_uri', 'album_uri'], dtype='object')

In [27]:
t_uri=df["track_uri"].unique()
a_uri=df["artist_uri"].unique()

# Feature extraction

Using the Spotify API for Feature Extraction and Saving Results to a CSV File and Errors to a Log File

I was using SP.track first, but I realised that it would take a lot of time and I would have to counter a lot of Api rate limits, so I used SP.tracks and SP.artists instead. They accept lists with a 50-URI maximum and handle them in a single request, so it took a lot less time.

In [29]:
f = open('data/audio_features.csv','a')
e=0
for i in tqdm(range(0,len(t_uri),100)):
    try:
     track_feature = sp.audio_features(t_uri[i:i+100])
     track_df = pd.DataFrame(track_feature)
     csv_data = track_df.to_csv(header=False,index=False)
     f.write(csv_data)
    except Exception as error:
        e+=1
        r = open("audio_features_log.txt", "a")
        r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+": "+str(error)+'\n')
        r.close()
        time.sleep(3)
        continue
r = open("audio_features_log.txt", "a")
r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+" _________________________ "+"Total Number Of Errors : "+str(e)+" _________________________ "+'\n')
r.close()
f.close()

  1%|▎                                        | 23/2960 [00:05<11:51,  4.13it/s]


KeyboardInterrupt: 

In [31]:
f = open('data/track_features.csv','a')
e=0
for i in tqdm(range(0,len(t_uri),50)):
    try:
        track_features = sp.tracks(t_uri[i:i+50])
        for x in range(50):
            track_pop=pd.DataFrame([t_uri[i+x]])
            track_pop['release_date']=track_features['tracks'][x]['album']['release_date']
            track_pop['pop'] = track_features['tracks'][x]["popularity"]
            csv_data = track_pop.to_csv(header=False,index=False)
            f.write(csv_data)
    except Exception as error:
        e+=1
        r = open("track_features.txt", "a")
        r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+": "+str(error)+'\n')
        r.close()
        time.sleep(3)
        continue
r = open("track_features.txt", "a")
r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+" _________________________ "+"Total Number Of Errors : "+str(e)+" _________________________ "+'\n')
r.close()
f.close()

  0%|                                                  | 0/5919 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [30]:
f = open('data/artist_features.csv','a')
e=0
for i in tqdm(range(0,len(a_uri),50)):
    try:
        artist_features = sp.artists(a_uri[i:i+50])
        for x in range(50):
            artist_df=pd.DataFrame([a_uri[i+x]])
            artist_pop = artist_features['artists'][x]["popularity"]
            artist_genres = artist_features['artists'][x]["genres"]
            artist_df["artist_pop"] = artist_pop
            if artist_genres: 
                artist_df["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
            else:
              artist_df["genres"] = "unknown"
            csv_data = artist_df.to_csv(header=False,index=False)
            f.write(csv_data)
    except Exception as error:
        e+=1
        r = open("artist_features.txt", "a")
        r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+": "+str(error)+'\n')
        r.close()
        time.sleep(3)
        continue
r = open("artist_features.txt", "a")
r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+" _________________________ "+"Total Number Of Errors : "+str(e)+" _________________________ "+'\n')
r.close()
f.close()

  0%|                                                  | 0/1108 [11:36<?, ?it/s]


KeyboardInterrupt: 